In [1]:
import warnings
warnings.filterwarnings('ignore')

import sklearn
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier

In [2]:
test = pd.read_csv('./Test Data.csv')
train = pd.read_csv('./Training Data.csv')

print(test.shape)
print(train.shape)

(28000, 12)
(252000, 13)


In [3]:
train = train.replace({'Married/Single': 'single'}, 0)
train = train.replace({'Married/Single': 'married'}, 1)
train = train.replace({'Car_Ownership': 'no'}, 0)
train = train.replace({'Car_Ownership': 'yes'}, 1)

test = test.replace({'Married/Single': 'single'}, 0)
test = test.replace({'Married/Single': 'married'}, 1)
test = test.replace({'Car_Ownership': 'no'}, 0)
test = test.replace({'Car_Ownership': 'yes'}, 1)

In [4]:
train

,Id,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
0,1,1303834,23,3,0,rented,0,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,0,rented,0,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,1,rented,0,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,0,rented,1,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,0,rented,0,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,0,rented,0,Surgeon,Kolkata,West_Bengal,6,11,0
251996,251997,2843572,26,10,0,rented,0,Army_officer,Rewa,Madhya_Pradesh,6,11,0
251997,251998,4522448,46,7,0,rented,0,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0
251998,251999,6507128,45,0,0,rented,0,Graphic_Designer,Pondicherry,Puducherry,0,10,0


In [5]:
column = ['Income', 'Age', 'Experience', 'Married/Single', 'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS']
X_train, X_test = train[column], test[column]
y_train = train['Risk_Flag']

In [6]:
# Encoding & Scale
ordinal_cols = ['House_Ownership']
standard_cols = ['Income', 'Age', 'Married/Single', 'Car_Ownership', 'CURRENT_JOB_YRS', 'CURRENT_HOUSE_YRS']
onehot_cols = ['Profession', 'CITY', 'STATE']

onehot=Pipeline([
    ('oridinal', OrdinalEncoder()),
    ('onehot', OneHotEncoder())
])

column_trans=ColumnTransformer([
    ('stadardard', StandardScaler(), standard_cols),
    ('ordinal', OrdinalEncoder(categories=[['norent_noown','rented','owned']]), ordinal_cols),
    ('onehot',  OneHotEncoder(handle_unknown = 'ignore'), onehot_cols)],
    remainder="passthrough"
)

In [7]:
scaled_X_train = column_trans.fit_transform(X_train)
scaled_X_test = column_trans.transform(X_test)

In [8]:
clf = XGBClassifier()
clf.fit(scaled_X_train, y_train)

[17:38:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
pred = clf.predict(scaled_X_test)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [10]:
sample = pd.read_csv('./Sample Prediction Dataset.csv')
answer = np.array(sample['risk_flag'])
answer

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [11]:
accuracy_score(answer, pred)

0.8665